In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [ ]:
# Load data
df0 = pd.read_csv('비트코인 과거 데이터 (1).csv',encoding='cp949')

In [ ]:
df0.head()

,날짜,종가,시가,고가,저가,거래량,변동 %
0,2023-10-30,34700,"34,526.70","34,853.30","34,149.50",41.03K,0.50%
1,2023-10-29,34527,"34,082.60","34,737.90","33,937.60",27.72K,1.30%
2,2023-10-28,34083,"33,901.60","34,387.80","33,867.30",22.00K,0.53%
3,2023-10-27,33902,"34,153.70","34,245.80","33,423.60",47.33K,-0.74%
4,2023-10-26,34154,"34,497.60","34,826.00","33,749.90",56.62K,-0.99%


1. df0에서 종가를 분리하여 df에 저장하고 열의 이름은 Close로 변경하시오.

In [ ]:
df=df0[['종가']].astype(int)
df.columns=['Close']

In [ ]:
df

,Close
0,34700
1,34527
2,34083
3,33902
4,34154
...,...
361,20206
362,20154
363,20484
364,20496


2. df를 이용하여 단변량 시계열 예측을 위해 필요한 종속변수와 독립변수를 만들어 각각 x_0와 x_1에 저장하시오.

In [ ]:
x_0 = np.array(df['Close'][0:-1])
x_1 = np.array(df['Close'][1:])

x = np.log(1-(x_1-x_0)/x_0)

In [ ]:
# Define hyperparameters
seq_len = 6
hidden_size = 5
num_layers = 1
learning_rate = 0.001
num_epochs = 100

3.위의 파라미터를 이용하여 rnn인스턴스를 선언하시오

In [ ]:
rnn = nn.RNN(input_size = 1, hidden_size = 5, num_layers = 1, batch_first = True)

In [ ]:
rnn

RNN(1, 5, batch_first=True)

4.batch_first의 옵션의 기능을 쓰시오.

- Batch_first = False 인 경우에 (seq, batch, feature) 로 input data x 를 입력한다.

- Batch_first = True 인 경우에 (batch, seq, feature) 로 input data x 를 입력한다.



5. torch.randn을 이용하여 h0를 만드시오. 차원만 맞게 만드시면 됩니다.

In [ ]:
h0 = torch.randn(1, 2, 5)
h0.dtype

torch.float32

6. __init__ 부분을 작성하시오

In [ ]:
# Define the RNN model
class myRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(myRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
# window size 설정
window_size = 6
# 입력 시퀀스와 출력 시퀀스 정의
def create_inout_sequences(input_data, window_size):
    inout_seq = []
    L = len(input_data)
    for i in range(L - window_size):
        train_seq = input_data[i:(i+window_size)].reshape(window_size,1)
        train_seq = train_seq.astype(np.float32)
        train_label = input_data[(i+window_size):(i+window_size+1)]
        train_label = train_label.astype(np.float32)
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
# 입력 시퀀스와 출력 시퀀스 생성
train_data = create_inout_sequences(x, window_size)
print('input:', train_data[0][0].shape)
print('output:', train_data[0][1].shape)
print('train_data:', len(train_data))

input: (6, 1)
output: (1,)
train_data: 359


 7. 데이터를 batch 단위로 나누시오. batch_size=10을 사용하시오

In [ ]:

batch_size = 10
train_loader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)



8. 빈칸을 작성하시오.
(평가 지표로 MSE를 사용하고 optimizer는 Adam을 사용하시오.)

In [ ]:

model = myRNN(1, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

9. epoch 10으로 설정하고 앞에서 설정한 조건을 이용하여 학습을 진행하시오. 각 epoch마다 loss를 출력하시오

In [ ]:

num_epochs = 10
for epoch in range(num_epochs):
  for i, (inputs, labels) in enumerate(train_loader):
    input = inputs.to(device)
    label = labels.to(device)
    output = model(input)
    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(loss)

tensor(0.0039, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0011, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0011, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0011, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0013, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0013, device='cuda:0', grad_fn=<MseLossBackward0>)
